Activation Functions in Keras

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, PReLU, ELU

model = Sequential()

# ReLU
model.add(Dense(64, activation='relu', input_dim=6))

# Leaky ReLU
model.add(Dense(64))
model.add(LeakyReLU(alpha=0.01))

# Parametric ReLU
model.add(Dense(64))
model.add(PReLU())

# ELU
model.add(Dense(64))
model.add(ELU(alpha=1.0))

# Output layer (Binary classification)
model.add(Dense(1, activation='sigmoid'))


c:\Users\HP\Desktop\deep learning\myenv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\HP\Desktop\deep learning\myenv\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Xavier & He Initialization

In [7]:
from tensorflow.keras.initializers import GlorotUniform, HeNormal

model = Sequential()

# Xavier (for tanh / sigmoid)
model.add(Dense(
    64,
    activation='tanh',
    kernel_initializer=GlorotUniform(),
    input_dim=6
))

# He (for ReLU)
model.add(Dense(
    64,
    activation='relu',
    kernel_initializer=HeNormal()
))

model.add(Dense(1, activation='sigmoid'))


Batch Normalization

In [8]:
from tensorflow.keras.layers import BatchNormalization

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=6))
model.add(BatchNormalization())

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))


SGD with Momentum

In [9]:
from tensorflow.keras.optimizers import SGD

optimizer = SGD(
    learning_rate=0.01,
    momentum=0.9
)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)


Nesterov Accelerated Gradient (NAG)

In [10]:
optimizer = SGD(
    learning_rate=0.01,
    momentum=0.9,
    nesterov=True
)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)


AdaGrad

In [11]:
from tensorflow.keras.optimizers import Adagrad

optimizer = Adagrad(learning_rate=0.01)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)


RMSProp, Adam & Keras Tuner

RMSProp

In [12]:
from tensorflow.keras.optimizers import RMSprop

optimizer = RMSprop(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)


Adam

In [13]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999
)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)


Keras Tuner – Hyperparameter Tuning

In [14]:
import keras_tuner as kt
print("Keras Tuner OK:", kt.__version__)


Keras Tuner OK: 1.4.8


In [16]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import HeNormal


In [17]:
# Load dataset
df = pd.read_csv("medium_classification_student_performance_1500.csv")

X = df.drop("Pass", axis=1)
y = df["Pass"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
# ANN Model
model = Sequential([
    Dense(32, activation="relu", kernel_initializer=HeNormal(),
          input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),

    Dense(16, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


c:\Users\HP\Desktop\deep learning\myenv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                │ (None, 32)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 865 (3.38 KB)

 Trainable params: 801 (3.13 KB)

 Non-trainable params: 64 (256.00 B)

evaluation

In [19]:
# Training
history = model.fit(
    X_train_scaled,
    y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=10,
    batch_size=32
)


Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7908 - loss: 0.4460 - val_accuracy: 0.9933 - val_loss: 0.2111
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9867 - loss: 0.1906 - val_accuracy: 1.0000 - val_loss: 0.1231
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0968 - val_accuracy: 1.0000 - val_loss: 0.0739
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0570 - val_accuracy: 1.0000 - val_loss: 0.0455
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0353 - val_accuracy: 1.0000 - val_loss: 0.0292
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0242 - val_accuracy: 1.0000 - val_loss: 0.0198
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0175 - val_accuracy: 1.0000 - val_loss: 0.0140
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0128 - val_accuracy: 1.0000 - val_loss:

In [20]:
# Evaluation
loss, acc = model.evaluate(X_test_scaled, y_test)
print("Test Accuracy:", acc)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0062 
Test Accuracy: 1.0
